# Data Preprocessing Tools

## Importing the libraries

In [182]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


## Importing the dataset

In [183]:
# Your selected columns
s4_req_cols = ['SAP Personnel #', 'Current_Role', 'Birthdate']
workday_req_cols = ['Employee_ID', 'Years_Of_Service', 'Last_Promoted', 'Department',
                    'Employee HR rate', '# of Hours per week', 'Calculated Column - Fully Loaded Cost']

# Load only required columns
df_s4 = pd.read_excel("0_raw_data/resources_s4.xlsx", usecols=s4_req_cols)
df_workday = pd.read_excel("0_raw_data/resources_workday.xlsx", usecols=workday_req_cols)


In [184]:
print(df_s4.head())
print(df_workday.head())

   SAP Personnel #               Current_Role  Birthdate
0               57   Senior Software Engineer 1991-05-19
1               58   Associate Data Scientist 1996-03-23
2               59  Associate Product Manager 1993-09-25
3               60           Business Analyst 1994-03-03
4               61    Chief Operating Officer 1984-01-13
   Employee_ID  Years_Of_Service Last_Promoted    Department  \
0           57               5.2    2022-08-15   Engineering   
1           58               1.7    2023-11-10  Data Science   
2           59               1.3    2024-03-05       Product   
3           60               3.9    2023-02-15       Finance   
4           61              15.2    2019-08-22     Executive   

   Employee HR rate  # of Hours per week  \
0                60                   40   
1               117                   40   
2               147                   40   
3               153                   40   
4                79                   40   

   Calcu

## Merging the dataset

In [185]:
# Merge on SAP Personnel # and Employee_ID
df_merged = pd.merge(df_s4, df_workday, left_on='SAP Personnel #', right_on='Employee_ID', how='inner')

# Drop both ID columns since they are just identifiers
df_merged.drop(columns=['SAP Personnel #'], errors='ignore', inplace=True)

In [186]:
print(df_merged.head())

                Current_Role  Birthdate  Employee_ID  Years_Of_Service  \
0   Senior Software Engineer 1991-05-19           57               5.2   
1   Associate Data Scientist 1996-03-23           58               1.7   
2  Associate Product Manager 1993-09-25           59               1.3   
3           Business Analyst 1994-03-03           60               3.9   
4    Chief Operating Officer 1984-01-13           61              15.2   

  Last_Promoted    Department  Employee HR rate  # of Hours per week  \
0    2022-08-15   Engineering                60                   40   
1    2023-11-10  Data Science               117                   40   
2    2024-03-05       Product               147                   40   
3    2023-02-15       Finance               153                   40   
4    2019-08-22     Executive                79                   40   

   Calculated Column - Fully Loaded Cost  
0                              163355.20  
1                               9292

In [187]:
print(df_merged.shape)

(100, 9)


## Feature Engineering

In [188]:
df_merged['Last_Promoted'] = pd.to_datetime(df_merged['Last_Promoted'])
df_merged['Birthdate'] = pd.to_datetime(df_merged['Birthdate'])
df_merged['Monthly FLC'] = df_merged['Calculated Column - Fully Loaded Cost']/12

today = pd.Timestamp.today()

# Calculate years since last promotion using days / 365.25
df_merged['Years_Since_Last_Promotion'] = (today - df_merged['Last_Promoted']).dt.days / 365.25

# Calculate age as integer years
df_merged['age'] = ((today - df_merged['Birthdate']).dt.days / 365.25).astype(int)


# Renaming DataFrame for feature-engineered data
df_featured = df_merged.copy()

# Optionally drop original columns
df_featured.drop(columns=['Last_Promoted', 'Birthdate'], inplace=True)

print(df_featured[['Years_Since_Last_Promotion', 'age']].head())

   Years_Since_Last_Promotion  age
0                    2.800821   34
1                    1.563313   29
2                    1.245722   31
3                    2.297057   31
4                    5.782341   41


In [189]:
print(df_featured.head())

                Current_Role  Employee_ID  Years_Of_Service    Department  \
0   Senior Software Engineer           57               5.2   Engineering   
1   Associate Data Scientist           58               1.7  Data Science   
2  Associate Product Manager           59               1.3       Product   
3           Business Analyst           60               3.9       Finance   
4    Chief Operating Officer           61              15.2     Executive   

   Employee HR rate  # of Hours per week  \
0                60                   40   
1               117                   40   
2               147                   40   
3               153                   40   
4                79                   40   

   Calculated Column - Fully Loaded Cost   Monthly FLC  \
0                              163355.20  13612.933333   
1                               92926.13   7743.844167   
2                              160081.53  13340.127500   
3                              105775.38

In [190]:
print(df_featured.columns)

Index(['Current_Role', 'Employee_ID', 'Years_Of_Service', 'Department',
       'Employee HR rate', '# of Hours per week',
       'Calculated Column - Fully Loaded Cost', 'Monthly FLC',
       'Years_Since_Last_Promotion', 'age'],
      dtype='object')


In [191]:
print(df_featured.shape)

(100, 10)


## Create Custom Dependent Column

In [192]:
# Create a boolean Series for condition
condition = df_featured['Years_Since_Last_Promotion'] > 5

# Group by that boolean condition
grouped = df_featured.groupby(condition)


In [193]:
print(grouped.size())

Years_Since_Last_Promotion
False    89
True     11
dtype: int64


In [194]:
df_featured['left'] = (df_featured['Years_Since_Last_Promotion'] > 5).astype(int)

# Renaming DataFrame for updated-engineered data
df_updated = df_featured.copy()

In [195]:
print(df_updated[['Years_Since_Last_Promotion', 'left']].head())

   Years_Since_Last_Promotion  left
0                    2.800821     0
1                    1.563313     0
2                    1.245722     0
3                    2.297057     0
4                    5.782341     1


## Export Updated Excel

In [196]:
# Create the folder if it doesn't exist
os.makedirs("1_updated_data", exist_ok=True)

# Save the Excel file inside the folder
df_updated.to_excel("1_updated_data/1_updated_data.xlsx", index=False)
